Topic Modelling with BERTopic

###### Documentation: https://maartengr.github.io/BERTopic/

<div style="color: light grey;">

- **Description**: BERTopic is a topic modeling technique that leverages transformers and c-TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions.
- **Purpose**: It is used in this project to understand the diversity of topics and their distribution across articles. It can be run on either the title or the whole article. Transformer models ensure quality for both short-length and full-length texts.
- **Deployment**: The results are deployed for further Sentiment Analysis and News Summarization tasks.
- **Input**: The raw BBC news dataset for the category 'Politics'.

</div>

In [7]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
# Import libraries for Topic Modelling with BERTopic
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
#from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
# local imports
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.query import bbc_news_politics
from src.preprocessing import summarize_long_articles, clean_text, drop_columns, filter_articles_by_length
# Setting secret credentials
from dotenv import load_dotenv #pip install python-dotenv
load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [2]:
# Fetch the data from BigQuery
df = bbc_news_politics()
df.head()

This query will process 5114621 bytes.


,body,title,filename,category
0,"The ""best person for the job"" should be appoin...",'Best person' for top legal job,bbc/politics/273.txt,politics
1,A cap on donations to political parties should...,'Debate needed' on donations cap,bbc/politics/059.txt,politics
2,A cap on donations to political parties should...,'Debate needed' on donations cap,bbc/politics/298.txt,politics
3,It could cost £80m to run a UK referendum on t...,'EU referendum could cost £80m',bbc/politics/391.txt,politics
4,The initial attempt to sell the Millennium Dom...,'Errors' doomed first Dome sale,bbc/politics/006.txt,politics


In [ ]:
# Preprocess the data
df = clean_text(df)
df = drop_columns(df)
df = filter_articles_by_length(df) # 150-1000 words

In [9]:
# Initialize BART summarization pipeline
summarization_pipeline = pipeline("summarization", model="facebook/bart-large-cnn")
# Summarize long articles (> 1000 words) and replace the original text with the summary
df = summarize_long_articles(df)  

Device set to use mps:0


In [10]:
df.head()

,body,title,category,word_count
0,"The ""best person for the job"" should be appoin...",'Best person' for top legal job,politics,465
1,A cap on donations to political parties should...,'Debate needed' on donations cap,politics,549
2,A cap on donations to political parties should...,'Debate needed' on donations cap,politics,549
3,It could cost £80m to run a UK referendum on t...,'EU referendum could cost £80m',politics,194
4,The initial attempt to sell the Millennium Dom...,'Errors' doomed first Dome sale,politics,380


In [11]:
# Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L12-v2")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anna_verbytska/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Device set to use mps:0


In [12]:
# Pre-compute embeddings
embeddings = embedding_model.encode(df['body'], show_progress_bar=False)

In [13]:
# Reduce dimensionality with UMAP
umap_model = UMAP(
    n_neighbors=10,  # capture more fine-grained relationships for a small dataset
    n_components=5,  # reduce dimensionality further for better visualization
    min_dist=0.0,  # keep clusters tight
    metric='cosine',  # best for text embeddings
    random_state=42
)

In [14]:
hdbscan_model = HDBSCAN(
    min_cluster_size=15,  # to allow more but smaller clusters
    min_samples=5,  # controls how "noise-tolerant" clustering is, detects smaller clusters
    metric='euclidean',  # distance metric
    cluster_selection_method='eom',  # extracts dense clusters
    prediction_data=True  # allows predicting cluster membership for new points
)

In [15]:
# Tokenize
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

In [16]:
# Create topic representation
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

In [17]:
# Set λ to 0.5 to get the optimal mix of diversity and accuracy in the result set
representation_model = MaximalMarginalRelevance(diversity=0.5, top_n_words=15)

In [18]:
# Set the parameters for the model
topic_model = BERTopic(
  embedding_model=embedding_model,            # extract embeddings
  umap_model=umap_model,                      # reduce dimensionality
  hdbscan_model=hdbscan_model,                # cluster reduced embeddings
  vectorizer_model=vectorizer_model,          # tokenize topics
  ctfidf_model=ctfidf_model,                  # extract topic words
  representation_model=representation_model,  # diversify topic words         
  nr_topics=None,                             # no forced merging of topics BUT nr_topics='auto' to merge similar ones
  min_topic_size=20,                          # filter out small, less meaningful topics
  verbose=True,
  top_n_words=15                              # number of words per topic
  )                            

In [19]:
# Initialize BERT model
topics, probabilities = topic_model.fit_transform(df['body'], embeddings)

2025-04-09 15:19:00,618 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2025-04-09 15:19:04,253 - BERTopic - Dimensionality - Completed ✓
2025-04-09 15:19:04,253 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-09 15:19:04,266 - BERTopic - Cluster - Completed ✓
2025-04-09 15:19:04,268 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-09 15:19:06,199 - BERTopic - Representation - Completed ✓


In [20]:
# Print the topics
freq = topic_model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq.head(15)   

Number of topics: 12


,Topic,Count,Name,Representation,Representative_Docs
0,-1,125,-1_labour_immigration_asylum_howard,"[labour, immigration, asylum, howard, tory, mi...",[Michael Howard has denied his shadow cabinet ...
1,0,48,0_party_students_mr kennedy_people,"[party, students, mr kennedy, people, educatio...",[Students can decide the fate of MPs in some s...
2,1,45,1_police_human rights_suspects_terror,"[police, human rights, suspects, terror, said,...",[UK citizens suspected of involvement in terro...
3,2,38,2_mr blair_mr brown_prime minister_prime,"[mr blair, mr brown, prime minister, prime, el...",[Gordon Brown has made an appeal for unity aft...
4,3,25,3_mr howard_tory_spending_party,"[mr howard, tory, spending, party, tax cuts, l...","[By any measure, New Yorker Dick Morris is tha..."
5,4,24,4_aid_g8_poverty_relief,"[aid, g8, poverty, relief, countries, mr brown...",[Tony Blair has had the chance to tackle the p...
6,5,20,5_workers_unions_pensions_public sector,"[workers, unions, pensions, public sector, str...",[Talks aimed at averting national strikes over...
7,6,19,6_straw_constitution_china_embargo,"[straw, constitution, china, embargo, turkey, ...",[French president Jacques Chirac has called th...
8,7,17,7_lord chancellor_courts_falconer_house lords,"[lord chancellor, courts, falconer, house lord...",[New rules on marriage for foreign nationals l...
9,8,16,8_ban_dogs_casinos_betting,"[ban, dogs, casinos, betting, foxes, hunting d...",[Hunts in England and Wales have begun on the ...


**Note:**  
All documents labeled as `-1` are considered **outliers**. This means they either:  
1. Contain **complex or ambiguous content**, making it difficult to assign a single topic.  
2. Lack distinct features, preventing clear topic classification.  

##### 🔹 How to Handle These Outliers?  
- **Option 1 (Recommended for Now):** Exclude `-1` documents from summarization and analysis to ensure topic clarity.  
- **Option 2 (Alternative Approach):** Assign them to the closest topic using similarity-based methods (e.g., cosine similarity with topic centroids).  

For now, we will proceed with **Option 1**, focusing only on clearly defined topics.

In [21]:
# Print the keywords
a_topic = freq.iloc[9]["Topic"] # select the topic by index
topic_model.get_topic(a_topic) # Show the words and their c-TF-IDF scores   

[('ban', 0.3720555448709897),
 ('dogs', 0.36032046292367537),
 ('casinos', 0.3589203726854613),
 ('betting', 0.3132777257719909),
 ('foxes', 0.31069527888200815),
 ('hunting dogs', 0.3039888027410621),
 ('countryside alliance', 0.2974680822535499),
 ('police', 0.27758224752118976),
 ('sports', 0.2761940452598711),
 ('industry', 0.2694238932686565),
 ('anti hunt', 0.26808773295277744),
 ('jowell', 0.26066282393807527),
 ('chasing', 0.2601590090698154),
 ('league cruel', 0.25521174068607083),
 ('hunting act', 0.25521174068607083)]

In [22]:
# Visualise the topics and their keywords
topic_model.visualize_barchart(n_words=10)

In [23]:
# Visualise clusters of topics
topic_model.visualize_topics()

In [24]:
# Visualise the topic hierarchy
topic_model.visualize_hierarchy(top_n_topics=11)

In [25]:

# Visualise a similarity matrix
topic_model.visualize_heatmap(top_n_topics=30)

In [26]:
# Create a new column filled with topics
df['topic'] = topics
df.head(5) 

,body,title,category,word_count,topic
0,"The ""best person for the job"" should be appoin...",'Best person' for top legal job,politics,465,7
1,A cap on donations to political parties should...,'Debate needed' on donations cap,politics,549,4
2,A cap on donations to political parties should...,'Debate needed' on donations cap,politics,549,-1
3,It could cost £80m to run a UK referendum on t...,'EU referendum could cost £80m',politics,194,6
4,The initial attempt to sell the Millennium Dom...,'Errors' doomed first Dome sale,politics,380,-1


In [27]:
# Select topics that are semantically similar to an input query
similar_topics, similarity = topic_model.find_topics('migration', top_n=2)
similar_topics

[-1, 9]